In [1]:
import torch

class TinyModel(torch.nn.Module):

    def __init__(self):
        super(TinyModel, self).__init__()

        self.linear1 = torch.nn.Linear(100, 200)
        self.activation = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(200, 10)
        self.softmax = torch.nn.Softmax()

    def forward(self, x):
        x = self.linear1(x)
        x = self.activation(x)
        x = self.linear2(x)
        x = self.softmax(x)
        return x

tinymodel = TinyModel()

print('The model:')
print(tinymodel)

print('\n\nJust one layer:')
print(tinymodel.linear2)

print('\n\nModel params:')
for param in tinymodel.parameters():
    print(param)

print('\n\nLayer params:')
for param in tinymodel.linear2.parameters():
    print(param)

The model:
TinyModel(
  (linear1): Linear(in_features=100, out_features=200, bias=True)
  (activation): ReLU()
  (linear2): Linear(in_features=200, out_features=10, bias=True)
  (softmax): Softmax(dim=None)
)


Just one layer:
Linear(in_features=200, out_features=10, bias=True)


Model params:
Parameter containing:
tensor([[-0.0329, -0.0056,  0.0100,  ..., -0.0954, -0.0940, -0.0308],
        [ 0.0436, -0.0163, -0.0449,  ...,  0.0976, -0.0716, -0.0569],
        [ 0.0178, -0.0235, -0.0987,  ...,  0.0420,  0.0105, -0.0995],
        ...,
        [-0.0274, -0.0294,  0.0312,  ...,  0.0520,  0.0328, -0.0732],
        [ 0.0309,  0.0208, -0.0637,  ..., -0.0384, -0.0449, -0.0659],
        [ 0.0268,  0.0843, -0.0537,  ..., -0.0120,  0.0637, -0.0064]],
       requires_grad=True)
Parameter containing:
tensor([-0.0314,  0.0973, -0.0123, -0.0086, -0.0842,  0.0788, -0.0480,  0.0679,
         0.0635, -0.0236,  0.0732,  0.0624,  0.0530, -0.0845, -0.0926, -0.0413,
        -0.0346, -0.0785,  0.0558, -0.04

In [2]:
lin = torch.nn.Linear(3, 2)
x = torch.rand(1, 3)
print('Input:')
print(x)

print('\n\nWeight and Bias parameters:')
for param in lin.parameters():
    print(param)

y = lin(x)
print('\n\nOutput:')
print(y)

Input:
tensor([[0.3322, 0.6636, 0.2999]])


Weight and Bias parameters:
Parameter containing:
tensor([[ 0.1570, -0.1894,  0.5639],
        [-0.4279,  0.0428,  0.0191]], requires_grad=True)
Parameter containing:
tensor([0.2297, 0.0031], requires_grad=True)


Output:
tensor([[ 0.3253, -0.1049]], grad_fn=<AddmmBackward0>)


In [3]:
import torch.functional as F


class LeNet(torch.nn.Module):

    def __init__(self):
        super(LeNet, self).__init__()
        # 1 input image channel (black & white), 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = torch.nn.Conv2d(1, 6, 5)
        self.conv2 = torch.nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = torch.nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = torch.nn.Linear(120, 84)
        self.fc3 = torch.nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [4]:
class LSTMTagger(torch.nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = torch.nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = torch.nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = torch.nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [5]:
my_tensor = torch.rand(1, 6, 6)
print(my_tensor)

maxpool_layer = torch.nn.MaxPool2d(3)
print(maxpool_layer(my_tensor))

tensor([[[0.1018, 0.4032, 0.0520, 0.7738, 0.1640, 0.3665],
         [0.1730, 0.9316, 0.0496, 0.1213, 0.5074, 0.4246],
         [0.2593, 0.5818, 0.6720, 0.0519, 0.9631, 0.0138],
         [0.9210, 0.6813, 0.4306, 0.1551, 0.2512, 0.9309],
         [0.3222, 0.8598, 0.2086, 0.5727, 0.2592, 0.7055],
         [0.7572, 0.7808, 0.9047, 0.0415, 0.0035, 0.3969]]])
tensor([[[0.9316, 0.9631],
         [0.9210, 0.9309]]])


In [6]:
my_tensor = torch.rand(1, 4, 4) * 20 + 5
print(my_tensor)

print(my_tensor.mean())

norm_layer = torch.nn.BatchNorm1d(4)
normed_tensor = norm_layer(my_tensor)
print(normed_tensor)

print(normed_tensor.mean())

tensor([[[13.3854, 10.1611, 13.1639, 13.7074],
         [10.4206,  5.9438, 13.8881,  8.5944],
         [17.4658, 12.0432, 22.3588, 23.9080],
         [17.6546,  7.7525, 16.9287, 22.3771]]])
tensor(14.3596)
tensor([[[ 0.5485, -1.7160,  0.3929,  0.7746],
         [ 0.2454, -1.3041,  1.4455, -0.3867],
         [-0.3186, -1.4873,  0.7360,  1.0699],
         [ 0.2788, -1.5912,  0.1417,  1.1706]]],
       grad_fn=<NativeBatchNormBackward0>)
tensor(7.4506e-09, grad_fn=<MeanBackward0>)


In [7]:
my_tensor = torch.rand(1, 4, 4)

dropout = torch.nn.Dropout(p=0.4)
print(dropout(my_tensor))
print(dropout(my_tensor))

tensor([[[0.1545, 1.3975, 0.0000, 0.8493],
         [1.6378, 0.0000, 0.0000, 0.0000],
         [0.2744, 1.1846, 0.1616, 0.0000],
         [1.3006, 0.8008, 1.5168, 1.3135]]])
tensor([[[0.0000, 1.3975, 0.5919, 0.8493],
         [1.6378, 0.9458, 0.7030, 0.0000],
         [0.2744, 1.1846, 0.1616, 0.0000],
         [1.3006, 0.8008, 1.5168, 1.3135]]])
